# Drums drums drums drums drums

Philly Joe Jones - "Billy Boy", from Miles Davis' *Milestones* (1958)

Transcribed by yours truly

In [57]:
import os

import mido
import music21

import jazzaiexperiments

In [47]:
# Comping
# tune_name = "billyboy_comping"
# input_filepath = "../data/midi/mine/billyboy/PhillyJoeJones_BillyBoy_Comping_Processed77.mid"

# Solo
tune_name = "billyboy_solo"
input_filepath = "../data/midi/mine/billyboy/PhillyJoeJones_BillyBoy_Solo_Processed77.mid"

Let's see if it works just training it in `single_melody` mode:

In [35]:
# model, note_events, input_filepath = jazzaiexperiments.lstm.train_on_midi_input(tune_name,
#                                                                                 mode="single_melody",
#                                                                                 input_filepath=input_filepath,
#                                                                                 num_epochs=1)
# model.summary()

Created 456 note events from ../data/midi/mine/billyboy/PhillyJoeJones_BillyBoy_Solo_Processed77.mid using mode single_melody
Formatted note data (446 seqs of length 10, 105 unique notes)
Created model
No weights loaded (to load weights, specify a `weights_filepath`)
Epoch 1/100
446/446 [==============================] - 4s - loss: 4.3520     
Epoch 2/100
446/446 [==============================] - 1s - loss: 3.7968     
Epoch 3/100
446/446 [==============================] - 1s - loss: 3.6885     
Epoch 4/100
446/446 [==============================] - 1s - loss: 3.6643     
Epoch 5/100
446/446 [==============================] - 1s - loss: 3.6386     
Epoch 6/100
446/446 [==============================] - 1s - loss: 3.6428     
Epoch 7/100
446/446 [==============================] - 1s - loss: 3.6083     
Epoch 8/100
446/446 [==============================] - 1s - loss: 3.6108     
Epoch 9/100
446/446 [==============================] - 2s - loss: 3.5416     
Epoch 10/100
446/446 [========

In [62]:
# model = jazzaiexperiments.lstm.load_model_weights(model, "../data/models/weights_billyboy_solo_20170706143300474703_99_0.6481.hdf5")
# model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 256)           264192    
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 105)               26985     
Total params: 816,489
Trainable params: 816,489
Non-trainable params: 0
_________________________________________________________________


In [36]:
# print("{} note events".format(len(note_events)))
# print("{} unique note events".format(len(jazzaiexperiments.midi.create_note_set(note_events))))


# # jazzaiexperiments.midi.create_note_set(note_events)[:10]
# [jazzaiexperiments.midi.note_event_to_dict(evt, "single_melody") for evt in jazzaiexperiments.midi.create_note_set(note_events)[:5]]

456 note events
105 unique note events


[{'noteoff_time': 32,
  'noteon_pitch': 36,
  'noteon_time': 16,
  'noteon_velocity': 40},
 {'noteoff_time': 64,
  'noteon_pitch': 36,
  'noteon_time': 1,
  'noteon_velocity': 50},
 {'noteoff_time': 32,
  'noteon_pitch': 36,
  'noteon_time': 16,
  'noteon_velocity': 50},
 {'noteoff_time': 48,
  'noteon_pitch': 36,
  'noteon_time': 16,
  'noteon_velocity': 50},
 {'noteoff_time': 48,
  'noteon_pitch': 36,
  'noteon_time': 48,
  'noteon_velocity': 50}]

In [40]:
# notes_out = jazzaiexperiments.lstm.generate_midi_output(model, note_events,
#                                                         num_notes_to_generate=100,
#                                                         time_multiplier=4,
#                                                         random_seed=False,
#                                                         add_seed_to_output=True,
#                                                         tune_name=tune_name,
#                                                         midi_source_filepath=input_filepath,
#                                                         data_dir="../data/output")
# notes_out[:20]

Constructed input sequence: [98, 100, 98, 100, 98, 97, 100, 98, 100, 98]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_billyboy_solo_20170706143845044721.mid


([(51, 60, 0, 48),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (51, 60, 0, 32),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (38, 50, 16, 32),
  (38, 70, 0, 48),
  (38, 70, 16, 32),
  (38, 60, 0, 48),
  (38, 60, 16, 32),
  (38, 80, 0, 48),
  (38, 60, 16, 32),
  (38, 60, 0, 48),
  (38, 60, 16, 32),
  (38, 60, 0, 96),
  (38, 60, 0, 48),
  (38, 60, 16, 48),
  (38, 50, 48, 32),
  (38, 60, 0, 87),
  (38, 60, 9, 32),
  (38, 60, 0, 8),
  (38, 60, 24, 48),
  (38, 60, 16, 8),
  (38, 60, 24, 32),
  (38, 60, 0, 88),
  (38, 60, 8, 40),
  (38, 60, 24, 48),
  (38, 50, 16, 7),
  (38, 50, 25, 32),
  (38, 60, 0, 88),
  (38, 60, 8, 40),
  (38, 60, 24, 48),
  (38, 60, 16, 7),
  (51, 60, 16, 32),
  (51, 70, 0, 48),
  (51, 60, 16, 32),
  (51, 50, 0, 48),
  (51, 60, 16, 32),
  (51, 60, 0, 32),
  (38, 50, 16, 32),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (51, 60, 0, 48),
  (51, 60, 16, 32),
  (51, 60

In [54]:
# jazzaiexperiments.midi.write_file(note_events, os.path.join("../data/output/", "{}_original.mid".format(tune_name)),
#                                   mode="single_melody",
#                                   time_multiplier=5,
#                                   midi_source_filepath=input_filepath)


'../data/output/billyboy_solo_original.mid'

... Not really. Among the issues are lack of **polyphony**, and the **timing** (which is tied to the `time_multiplier`, which has to be manually set every time, which is bad)

### Fixing polyphony

One problem seems to be that the `single_melody` mode can't really handle notes that occur simultaneously. Thus we have to build in some kind of polyphony handling.

Functions to update:
- midi.extract_note_pairs
- midi.normalize_velocities
- midi.create_note_events
- lstm.train_on_midi_input

In [56]:
from jazzaiexperiments import *

In [113]:
midi_file = mido.MidiFile(input_filepath)
midi_track = midi_file.tracks[0]
[msg for msg in midi_track[:100]]

[<meta message track_name name='\x00' time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>,
 <message control_change channel=0 control=91 value=19 time=0>,
 <message control_change channel=0 control=91 value=0 time=0>,
 <message control_change channel=0 control=91 value=0 time=768>,
 <message control_change channel=0 control=91 value=19 time=0>,
 <message note_on channel=0 note=51 velocity=66 time=0>,
 <message note_off channel=0 note=51 velocity=0 time=48>,
 <message note_on channel=0 note=38 velocity=57 time=16>,
 <message note_on channel=0 note=44 velocity=74 time=32>,
 <message note_on channel=0 note=51 velocity=74 time=0>,
 <message note_off channel=0 note=38 velocity=0 time=16>,
 <message note_off channel=0 note=44 velocity=0 time=32>,
 <message note_off channel=0 note=51 velocity=0 time=0>,
 <mess

In [75]:
# From lstm.train_on_midi_input()
midi_track = midi.load_melody_from_file(input_filepath)
note_pairs = midi.extract_note_pairs(midi_track)
note_pairs = midi.normalize_velocities(note_pairs, interval=10)
print("Track has {} note messages; {} note pairs created".format(len(midi.extract_note_messages(midi_track)), len(note_pairs)))
print("... which means we're missing {} notes".format((len(midi.extract_note_messages(midi_track))/2) - len(note_pairs)))

Track has 2086 note messages; 456 note pairs created
... which means we're missing 587.0 notes


So let's redefine our `extract_note_pairs` function:

In [79]:
note_pairs[0][-1]

<message note_off channel=0 note=51 velocity=0 time=48>

In [164]:
def extract_note_pairs(track, enable_polyphony=False):
    """Extract note on/off pairs from a MIDI track.
    
    If polyphony is enabled, each pair will actually consist of a list
    of note ons and a list of note offs in the form (noteons, noteoffs).
    """
    notes = midi.extract_note_messages(track)
    note_pairs = []
    if enable_polyphony:
        note_pair = ([], [])
        for note in notes:
            if note.type == "note_on":
                note_ons = note_pair[0]
                note_offs = note_pair[1]
                if len(note_ons) < 1 or note.time == 0:
                    note_ons.append(note)
                    note_pair = (note_ons, note_offs)
                else:
                    for existing_note in note_ons:
                        note_off = mido.Message("note_off",
                                                channel=existing_note.channel,
                                                note=existing_note.note,
                                                velocity=existing_note.velocity,
                                                time=note.time)
                        note_offs.append(note_off)
                    note_pair = (note_ons, note_offs)
                    note_pairs.append(note_pair)
                    note_pair = ([], [])
            elif note.type == "note_off":
                note_ons = note_pair[0]
                note_offs = note_pair[1]
                if len(note_ons) > 0:
                    for existing_note in note_ons:
                        note_off = mido.Message("note_off",
                                                channel=existing_note.channel,
                                                note=existing_note.note,
                                                velocity=existing_note.velocity,
                                                time=note.time)
                        note_offs.append(note_off)
                    note_pair = (note_ons, note_offs)
                    note_pairs.append(note_pair)
                    note_pair = ([], [])
    else:
        note_pairs = [(notes[i], notes[i + 1]) for i, _ in enumerate(notes[:-1])
                      if notes[i].type == "note_on" and
                      notes[i + 1].type == "note_off" and
                      notes[i].note == notes[i + 1].note]
    return note_pairs

In [168]:
def normalize_velocities(note_pairs, interval=10):
    """Normalize note velocities."""
    for i, note_pair in enumerate(note_pairs):
        note_ons, note_offs = note_pair
        
        if type(note_ons) is list:
            for note_on in note_ons:
                note_on.velocity = note_on.velocity - (note_on.velocity % interval)
        else:
            note_on = note_ons
            note_on.velocity = note_on.velocity - (note_on.velocity % interval)
        
        note_pair = (note_ons, note_offs)
        note_pairs[i] = note_pair
                
    return note_pairs

In [169]:
midi_track = midi.load_melody_from_file(input_filepath)
note_pairs = extract_note_pairs(midi_track, enable_polyphony=True)
note_pairs = normalize_velocities(note_pairs, interval=10)
print(len([(len(note_pair[0]), len(note_pair[1])) for note_pair in note_pairs if len(note_pair[0]) > 1]))
[(len(note_pair[0]), len(note_pair[1])) for note_pair in note_pairs if len(note_pair[0]) > 1][:10]

81


[(2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (3, 3),
 (2, 2),
 (2, 2),
 (2, 2)]

In [174]:
[(i, len(note_pair[0]), len(note_pair[1])) for i, note_pair in enumerate(note_pairs)][:20]

[(0, 1, 1),
 (1, 1, 1),
 (2, 1, 1),
 (3, 1, 1),
 (4, 1, 1),
 (5, 1, 1),
 (6, 1, 1),
 (7, 1, 1),
 (8, 1, 1),
 (9, 1, 1),
 (10, 1, 1),
 (11, 1, 1),
 (12, 1, 1),
 (13, 1, 1),
 (14, 1, 1),
 (15, 2, 2),
 (16, 1, 1),
 (17, 1, 1),
 (18, 1, 1),
 (19, 2, 2)]

In [171]:
len(note_pairs)

803

In [172]:
note_pairs[:10]

[([<message note_on channel=0 note=51 velocity=60 time=0>],
  [<message note_off channel=0 note=51 velocity=66 time=48>]),
 ([<message note_on channel=0 note=38 velocity=50 time=16>],
  [<message note_off channel=0 note=38 velocity=57 time=32>]),
 ([<message note_on channel=0 note=51 velocity=70 time=0>],
  [<message note_off channel=0 note=51 velocity=74 time=16>]),
 ([<message note_on channel=0 note=51 velocity=70 time=16>],
  [<message note_off channel=0 note=51 velocity=71 time=32>]),
 ([<message note_on channel=0 note=51 velocity=60 time=0>],
  [<message note_off channel=0 note=51 velocity=62 time=96>]),
 ([<message note_on channel=0 note=44 velocity=60 time=0>],
  [<message note_off channel=0 note=44 velocity=66 time=0>]),
 ([<message note_on channel=0 note=51 velocity=60 time=0>],
  [<message note_off channel=0 note=51 velocity=66 time=48>]),
 ([<message note_on channel=0 note=51 velocity=60 time=16>],
  [<message note_off channel=0 note=51 velocity=66 time=32>]),
 ([<message no

Alright, let's see if that works with the rest of the input pipeline:

In [185]:
def create_note_events(note_pairs, mode="single_melody", chords=[]):
    """Create note events from note pairs.

    This is the base data structure for note manipulation.
    """
    note_events = []
    if mode == "single_melody":
        note_events = [(note_on.note, note_on.velocity,
                        note_on.time, note_off.time)
                       for note_on, note_off in note_pairs]
    elif mode == "single_melody_harmony":
        if len(chords) < len(note_pairs):
            print("ERROR: Number of chords must match number of melody notes!")
            return note_events

        note_events = [(note_on.note, note_on.velocity,
                        note_on.time, note_off.time,
                        chords[i])
                       for i, (note_on, note_off) in enumerate(note_pairs)]
    elif mode == "drums":
        for note_pair in note_pairs:
            note_ons = note_pair[0]
            note_offs = note_pair[1]
            
            note_on_list = [(note_on.note, note_on.velocity) for note_on in note_ons]
            note_on_list = sorted(note_on_list, key=lambda x: x[0])
            note_on_time = note_ons[0].time
            note_off_time = note_offs[0].time
            # print([note_off.time for note_off in note_offs])  # Should all be equal

            note_event = (note_on_list, note_on_time, note_off_time)
            note_events.append(note_event)

    return note_events

In [191]:
note_events = create_note_events(note_pairs, mode="drums")
# note_set = midi.create_note_set(note_events)
# len(note_set)

In [192]:
note_events[0]

([(51, 60)], 0, 48)

### Fixing polyphony, take 2

Let's try this again. This time we'll just try to get the soonest note off for each note on. And since the time delta for certain note ons will be 0, this should achieve the same effect as manually handling polyphony.

Functions to update:
- midi.extract_note_pairs

In [216]:
def extract_note_pairs(track):
    """Extract note on/off pairs from a MIDI track."""
    notes = midi.extract_note_messages(track)
    note_pairs = []
    
    # TODO: Test that this method still works for the old modes
    # `single_melody` and `single_melody_harmony`
    for i, note in enumerate(notes):
        if note.type == "note_on":
            # Register our note on
            note_on = note
            
            # Find the earliest subsequent note off for this note on,
            # and then create a note pair out of it
            for other_note in notes[i:]:
                if other_note.type == "note_off" \
                and other_note.note == note.note \
                and other_note.time != 0:
                    note_off = other_note
                    note_pairs.append((note_on, note_off))
                    break
    
    # Old method where we don't look beyond the note immediately following
    # the note on event
    # note_pairs = [(notes[i], notes[i + 1]) for i, _ in enumerate(notes[:-1])
    #               if notes[i].type == "note_on" and
    #               notes[i + 1].type == "note_off" and
    #               notes[i].note == notes[i + 1].note]

    return note_pairs

In [217]:
midi_track = midi.load_melody_from_file(input_filepath)
note_pairs = extract_note_pairs(midi_track)
note_pairs = midi.normalize_velocities(note_pairs, interval=10)
len(note_pairs)

1024

In [218]:
note_pairs[:10]

[(<message note_on channel=0 note=51 velocity=60 time=0>,
  <message note_off channel=0 note=51 velocity=0 time=48>),
 (<message note_on channel=0 note=38 velocity=50 time=16>,
  <message note_off channel=0 note=38 velocity=0 time=16>),
 (<message note_on channel=0 note=44 velocity=70 time=32>,
  <message note_off channel=0 note=44 velocity=0 time=32>),
 (<message note_on channel=0 note=51 velocity=70 time=0>,
  <message note_off channel=0 note=51 velocity=0 time=32>),
 (<message note_on channel=0 note=51 velocity=70 time=16>,
  <message note_off channel=0 note=51 velocity=0 time=32>),
 (<message note_on channel=0 note=38 velocity=60 time=32>,
  <message note_off channel=0 note=38 velocity=0 time=16>),
 (<message note_on channel=0 note=51 velocity=60 time=0>,
  <message note_off channel=0 note=51 velocity=0 time=32>),
 (<message note_on channel=0 note=36 velocity=60 time=96>,
  <message note_off channel=0 note=36 velocity=0 time=48>),
 (<message note_on channel=0 note=44 velocity=60 ti

In [219]:
# Number of notes that immediately follow the previous note
len([1 for note_pair in note_pairs if note_pair[0].time == 0])

411

In [220]:
note_events = midi.create_note_events(note_pairs, mode="single_melody")
note_set = midi.create_note_set(note_events)
len(note_set)

248

OK brb let's add this to the module!

In [229]:
import importlib
importlib.reload(jazzaiexperiments)

<module 'jazzaiexperiments' from '/Users/usdivad/.pyenv/versions/3.5.1/lib/python3.5/site-packages/jazzaiexperiments-0.0.1-py3.5.egg/jazzaiexperiments/__init__.py'>

In [234]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
jazzaiexperiments.midi.extract_note_pairs??

### Fixing MIDI timing

So that we can FINALLY eliminate the need for the `time_multiplier`..